In [97]:
from neo4j import GraphDatabase
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50)

In [2]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [3]:
# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "12345678")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

conn = Neo4jConnection(uri=URI, user=AUTH[0], pwd=AUTH[1])

Find the top 3 most cited papers of each conference:

In [32]:
query =''' MATCH (c:Conference)
CALL {
    WITH c
    MATCH (p1:Paper)<-[:CITES]-(p2:Paper)-[:PRESENTED_IN]->(:Edition)--(c1:Conference)
    WHERE c.id = c1.id
    WITH p1, c, count(*) as cites
    ORDER BY c, cites DESC LIMIT 3
    RETURN p1.title AS tit,  cites AS cit
}
RETURN c.id, tit, cit
'''
res = conn.query(query=query)
df = pd.DataFrame(res, columns=["Conference", "Paper Title", "Number of Citations"])
df

Conference                                        Paper Title  \
0         ACL  A Formal Operational Semantics for PROLOG III:...   
1         ACL  Ibo van de Poel and David E. Goldberg (eds): P...   
2         ACL  Spezifikation eines Theorembeweisers für dreiw...   
3        ICML  A Lagrangian Method for Solving the Optimal Ou...   
4        ICML                     Inheritance for ADTs (revised)   
5        ICML     Object ADTs (with improvements for Value ADTs)   
6        CVPR  Differences in productivity and impact across ...   
7        CVPR  On Identifying Plausibility and Deliberative P...   
8        CVPR  On Boolean vs. Modular Arithmetic for Circuits...   
9     NeurIPS  Performance Analysis of SDN Specific Error Pro...   
10    NeurIPS  Sustainable Development as a Challenge for Und...   
11    NeurIPS     A Logical Operational Semantics of Full Prolog   
12       AAAI   An Approach to Manage Large Inheritance Networks   
13       AAAI  Darurah (Necessity) and Its Application in Isl...   
14       AAAI  A Tale of Two Perspectives: Regulation Versus ...   

    Number of Citations  
0                     3  
1                     3  
2                     2  
3                     3  
4                     3  
5                     3  
6                     4  
7                     2  
8                     2  
9                     3  
10                    3  
11                    3  
12                    3  
13                    3  
14                    3

For each conference find its community: i.e., those authors that have published papers
on that conference in, at least, 4 different editions.

In [41]:
query =''' MATCH (c:Conference)
CALL {
    WITH c
    MATCH (aut:Author)-[:WROTE]->(p:Paper)-[:PRESENTED_IN]->(e:Edition)--(c1:Conference)
    WHERE c.id = c1.id
    WITH aut, c, count(e) as num_editions
    ORDER BY c, aut, num_editions DESC
    WHERE num_editions >= 4
    RETURN aut.name AS author_name,  num_editions 
}
RETURN c.id, author_name, num_editions
'''
res = conn.query(query=query)
df = pd.DataFrame(res, columns=["Conference", "Author Name", "Number of Editions"])
df

Conference         Author Name  Number of Editions
0        ACL        Jochen Dörre                   4
1       ICML    Christoph Meinel                   6
2       CVPR    Christoph Meinel                   7
3    NeurIPS   Christoph Beierle                   4
4    NeurIPS         Dieter Baum                   4
5    NeurIPS    Christoph Meinel                   6
6    NeurIPS     Peter Gritzmann                   4
7    NeurIPS  Rainer Tichatschke                   4

Find the impact factors of the journals in your graph (see https://en.wikipedia.org/wiki/Impact_factor, for the definition of the impact factor).

In [78]:
query =''' MATCH (j:Journal)
UNWIND [1990, 1999, 2000, 2001, 2002, 2003, 2004, 2018] as year
CALL {
    WITH j, year
    MATCH (p1:Paper)-[cit:CITES]->(p2:Paper)-[:BELONGS_TO]->(v:Volume)-[pub:PUBLISHED_IN]-(j1:Journal)
    WHERE j.id = j1.id AND pub.year = year
    WITH j, year, count(cit) AS num_citations
    ORDER BY j, num_citations
    CALL{
        WITH j, year
        MATCH (p3:Paper)-[:BELONGS_TO]->(v:Volume)-[pub1:PUBLISHED_IN]-(j2:Journal)
        WHERE j.id = j2.id AND year-3 < pub1.year < year
        WITH j, count(p3) AS num_publications
        RETURN num_publications
    }
    RETURN num_citations, num_publications
}
RETURN j.name, j.id, year, num_citations, num_publications, toFloat(num_citations)/num_publications
'''
res = conn.query(query=query)
df = pd.DataFrame(res, columns=["Journal", "ID", "Year", "Number Citations", "Number Publications", "Impact Factor"])
df

Journal        ID  \
0                                GTE Laboratories Incorporated  49371352   
1                                                  IWBS Report  76456240   
2  Universität Trier, Mathematik/Informatik, Forschungsbericht  33940101   
3  Universität Trier, Mathematik/Informatik, Forschungsbericht  33940101   
4  Universität Trier, Mathematik/Informatik, Forschungsbericht  33940101   
5  Universität Trier, Mathematik/Informatik, Forschungsbericht  33940101   
6  Universität Trier, Mathematik/Informatik, Forschungsbericht  33940101   
7  Universität Trier, Mathematik/Informatik, Forschungsbericht  33940101   
8                                             Sci. Eng. Ethics  26291782   

   Year  Number Citations  Number Publications  Impact Factor  
0  1990                 1                    2       0.500000  
1  1990               106                   34       3.117647  
2  1999                86                   57       1.508772  
3  2000                36                   59       0.610169  
4  2001                58                   42       1.380952  
5  2002                64                   32       2.000000  
6  2003                17                   42       0.404762  
7  2004                23                   29       0.793103  
8  2018               126                   79       1.594937

Find the h-indexes of the authors in your graph (see https://en.wikipedia.org/wiki/H-index, for a definition of the h-index metric).

In [98]:
query =''' MATCH (a:Author)-[:WROTE]->(p1:Paper)<-[:CITES]-(p2:Paper)
WITH a, p1, count(*) as num_citations 
ORDER BY num_citations DESC
WITH a, collect(num_citations) as list_num_citations
WITH a, list_num_citations, [x in range(0, size(list_num_citations)-1) WHERE x < list_num_citations[x] | x+1] AS res
RETURN a.name, res[-1]
'''
res = conn.query(query=query)
df = pd.DataFrame(res, columns=["Author Name", "H-index"])
df

Author Name  H-index
0      Stefan Benzschawel        2
1              E. Levitin        2
2            Oliver Stein        1
3         Jan-J. Rückmann        1
4     Hubertus Th. Jongen        1
...                   ...      ...
1225           Lamia Kara        1
1226      Nassim Kazitani        1
1227  Noureddine Bettahar        1
1228  Mustapha Benmansour        1
1229      Daniele Fanelli        1

[1230 rows x 2 columns]

QUESTIONS:
- Years list in Impact Factor query.
- Add attribut of number of citations in papers
- Inverse edges
- Do have to prepare queries to easily update the graph (thinking how to delete the right edges for example)

In [99]:
conn.close()